In [1]:
%load_ext bigquery_magics



In [2]:
%%bigquery stations_ok
WITH base AS (
  SELECT station_number, year
  FROM `bigquery-public-data.samples.gsod`
  WHERE station_number BETWEEN 725300 AND 725330
    AND year BETWEEN 2000 AND 2005
  GROUP BY station_number, year
),
per_station AS (
  SELECT station_number, COUNT(DISTINCT year) AS years_present
  FROM base
  GROUP BY station_number
)
SELECT station_number
FROM per_station
WHERE years_present = 6
ORDER BY station_number

# this query filters stations that have atleast one row for every year 2000-2005

Query is running:   0%|          |

Downloading:   0%|          |

In [3]:
stations_ok

,station_number
0,725300
1,725305
2,725314
3,725315
4,725316
5,725317
6,725320
7,725326
8,725327
9,725330


In [4]:
from pathlib import Path
Path("data/interim").mkdir(parents=True, exist_ok=True)
stations_ok.to_csv("data/interim/stations_ok.csv", index=False)


In [5]:


%%bigquery snowfall_by_year
WITH station_ok AS (
  WITH base AS (
    SELECT station_number, year
    FROM `bigquery-public-data.samples.gsod`
    WHERE station_number BETWEEN 725300 AND 725330
      AND year BETWEEN 2000 AND 2005
    GROUP BY station_number, year
  ),
  per_station AS (
    SELECT station_number, COUNT(DISTINCT year) AS years_present
    FROM base
    GROUP BY station_number
  )
  SELECT station_number
  FROM per_station
  WHERE years_present = 6
)
SELECT
  year,
  SUM(CAST(snow AS INT64)) AS snow_day_count
FROM `bigquery-public-data.samples.gsod`
WHERE year BETWEEN 2000 AND 2005
  AND station_number IN (SELECT station_number FROM station_ok)
GROUP BY year
ORDER BY snow_day_count DESC, year


# This query filters stations 725300–725330 that appear in every year 2000–2005, 
# then sums the number of snow days per year across those stations and lists years sorted by highest snow-day count
# Assuming “Most snowfall” here means number of days with reported snow (frequency )

Query is running:   0%|          |

Downloading:   0%|          |

In [6]:
snowfall_by_year

,year,snow_day_count
0,2005,826
1,2000,797
2,2001,759
3,2002,745
4,2003,555
5,2004,262


In [7]:

%%bigquery depth_changes
WITH station_ok AS (
  WITH base AS (
    SELECT station_number, year
    FROM `bigquery-public-data.samples.gsod`
    WHERE station_number BETWEEN 725300 AND 725330
      AND year BETWEEN 2000 AND 2005
    GROUP BY station_number, year
  ),
  per_station AS (
    SELECT station_number, COUNT(DISTINCT year) AS years_present
    FROM base
    GROUP BY station_number
  )
  SELECT station_number
  FROM per_station
  WHERE years_present = 6
),
filtered AS (
  SELECT
    station_number,
    DATE(year, month, day) AS dte,
    snow_depth
  FROM `bigquery-public-data.samples.gsod`
  WHERE year BETWEEN 2000 AND 2005
    AND station_number IN (SELECT station_number FROM station_ok)
),
with_change AS (
  SELECT
    station_number,
    dte,
    snow_depth,
    snow_depth - LAG(snow_depth) OVER (
      PARTITION BY station_number
      ORDER BY dte
    ) AS snow_depth_change
  FROM filtered
)
SELECT
  station_number,
  FORMAT_DATE('%Y-%m-%d', dte) AS date_ymd,
  snow_depth,
  snow_depth_change
FROM with_change
ORDER BY station_number, dte


# This query selects stations from 2000–2005, 
# then for each such station computes daily snow_depth and its daily change using LAG


Query is running:   0%|          |

Downloading:   0%|          |

In [8]:
depth_changes


,station_number,date_ymd,snow_depth,snow_depth_change
0,725300,2000-01-01,NaN,NaN
1,725300,2000-01-02,NaN,NaN
2,725300,2000-01-03,NaN,NaN
3,725300,2000-01-04,1.2,NaN
4,725300,2000-01-05,NaN,NaN
...,...,...,...,...
21848,725330,2005-12-27,NaN,NaN
21849,725330,2005-12-28,NaN,NaN
21850,725330,2005-12-29,NaN,NaN
21851,725330,2005-12-30,NaN,NaN


In [9]:
%%bigquery max_increase
WITH station_ok AS (
  WITH base AS (
    SELECT station_number, year
    FROM `bigquery-public-data.samples.gsod`
    WHERE station_number BETWEEN 725300 AND 725330
      AND year BETWEEN 2000 AND 2005
    GROUP BY station_number, year
  ),
  per_station AS (
    SELECT station_number, COUNT(DISTINCT year) AS years_present
    FROM base
    GROUP BY station_number
  )
  SELECT station_number
  FROM per_station
  WHERE years_present = 6
),
filtered AS (
  SELECT
    station_number,
    DATE(year, month, day) AS dte,
    snow_depth
  FROM `bigquery-public-data.samples.gsod`
  WHERE year BETWEEN 2000 AND 2005
    AND station_number IN (SELECT station_number FROM station_ok)
),
with_change AS (
  SELECT
    station_number,
    dte,
    snow_depth - LAG(snow_depth) OVER (
      PARTITION BY station_number
      ORDER BY dte
    ) AS snow_depth_change
  FROM filtered
)
SELECT
  station_number,
  FORMAT_DATE('%Y-%m-%d', dte) AS date_ymd,
  snow_depth_change
FROM with_change
WHERE snow_depth_change IS NOT NULL
ORDER BY snow_depth_change DESC
LIMIT 1

# This query finds the single largest positive daily increase in snow_depth across stations with 2000–2005 coverage,
# returning the station, date, and change value using LAG over station/date.



Query is running:   0%|          |

Downloading:   0%|          |

In [10]:
max_increase

,station_number,date_ymd,snow_depth_change
0,725300,2005-01-22,9.8
